In [165]:
import pandas as pd
import plotly.express as px
import statsmodels.api as sm

In [166]:
df = pd.read_excel(r"C:\Users\mikej\Desktop\fantrax_season_data.xlsm")
df.columns

Index(['team', 'players_started', 'pars_num', 'bird_num', 'eag_num', 'bog_num',
       'dbog_num', 'alb_num', 'h1_num', 'other_num', 'plc_total', 'week',
       'win_loss', 'opponent', 'fin_1', 'fin_2', 'fin_3', 'fin_4', 'fin_5',
       'fin_6', 'median', 'cuts_made', 'median_delta', 'fin_1_pts',
       'fin_2_pts', 'fin_3_pts', 'fin_4_pts', 'fin_5_pts', 'fin_6_pts',
       'pars_pts', 'bird_pts', 'eag_pts', 'bog_pts', 'dbog_pts', 'alb_pts',
       'h1_pts', 'other_pts', 'plc_pts', 'total_pts', 'total_holes',
       'pp_hole'],
      dtype='object')

In [167]:
team_color={
                "Philly919": 'rgb(127,60,141)',
                "unit_cirle": 'rgb(17,165,121)',
                "AlphaWired": 'rgb(57,105,172)',
                "Snead's Foot": 'rgb(242,183,1)',
                "New Team 4": 'rgb(231,63,116)',
                "Team Gamble": 'rgb(230,131,16)',
                "txmoonshine": 'rgb(0,134,139)',
                "Putt Pirates": 'rgb(165,170,153)'}

In [168]:
team_medians = pd.DataFrame(df.groupby('team',as_index=False)['median_delta'].sum()).sort_values(by='median_delta',ascending=False).reset_index(drop=True)

median_delta_bar = px.bar(team_medians,
                          text_auto='.2s',
                          color='team',
                          color_discrete_map=team_color,
                          title='Total Pts vs. Weekly Median<br>thru 8 Weeks',
                          template='plotly_white',
                          labels={'index':'', 'value':''}
                         )

median_delta_bar.update_layout(title_x=.5,
                               legend=dict(
                                   title=None,
#                                    orientation='h',
#                                    x=0,
#                                    y=0
                               )
                              )

median_delta_bar.update_xaxes(showticklabels=False)
median_delta_bar.update_yaxes(showticklabels=False, showgrid=False)
median_delta_bar.update_layout(hoverlabel=dict(font_size=18,font_family="Rockwell"),
                              title_x=.45,
                              showlegend=True,
#                               legend=dict(orientation='h',
#                                           yanchor="bottom",
#                                           y=-.1,
#                                           xanchor="center",
#                                           x=.5,
#                                           title='')
                              )

median_delta_bar.show()

In [169]:
# descriptive stats on main categories

round(df[['pars_num','bird_num','eag_num','bog_num','dbog_num','alb_num','h1_num','plc_total']].describe().T.set_axis(['Par','Birdie','Eagle','Bogey','D Bogey','Albatross','Hole-in-One','Place']))

,count,mean,std,min,25%,50%,75%,max
Par,64.0,223.0,33.0,137.0,198.0,224.0,238.0,284.0
Birdie,64.0,84.0,20.0,42.0,71.0,80.0,95.0,136.0
Eagle,64.0,3.0,2.0,0.0,2.0,3.0,4.0,7.0
Bogey,64.0,42.0,12.0,17.0,33.0,42.0,49.0,73.0
D Bogey,64.0,4.0,3.0,0.0,2.0,4.0,6.0,12.0
Albatross,64.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
Hole-in-One,64.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
Place,64.0,241.0,70.0,83.0,196.0,240.0,293.0,412.0


In [205]:
# weekly scores showing wins/losses
temp_df = df.copy()
temp_df['win_loss'] = temp_df['win_loss'].astype('category')
scatter_fig = px.scatter(temp_df,
                        x='week',
                        y='total_pts',
                        color='win_loss',
                        height=600,
                        size='cuts_made',
                        size_max=16,
                        template='plotly_white',
                        hover_name='team',
                        title='Weekly Scores by Winner/Loser',
                        color_discrete_sequence=px.colors.qualitative.Pastel1,
#                         log_y=True
                        )

scatter_fig.update_xaxes(tickangle= -45,
                         tickvals = [1,2,3,4,5,6,7,8],
                         ticktext = ['Sony','Amex','Farmers','AT&T','Waste Mgmt','Genesis','Mexico Open','Cognizant'])

scatter_fig.update_traces(marker=dict(
#     size=12,
    line_color='black',
    opacity=.8))

scatter_fig.show()

In [40]:
finish_medians = round(df[['team','fin_1','fin_2','fin_3','fin_4','fin_5','fin_6']].groupby('team').median(),1).reset_index()
finish_medians.columns = 'team','Top Finisher','2nd','3rd','4th','5th','Worst Finisher'
finish_medians.head(2)

,team,Top Finisher,2nd,3rd,4th,5th,Worst Finisher
0,AlphaWired,7.5,11.0,26.0,49.0,65.0,69.0
1,New Team 4,8.0,13.0,44.0,66.0,68.0,70.0


In [41]:
team_stat_medians = round(df[['team','total_pts','cuts_made','pars_pts','bird_pts','eag_pts','bog_pts','dbog_pts','plc_pts']].groupby('team').median(),1).reset_index()
team_stat_medians.columns = 'team','Total Pts','Median Cuts Made','Pars','Birdies','Eagles','Bogeys','Doubles','Place Median'
team_stat_medians.head(2)

,team,Total Pts,Median Cuts Made,Pars,Birdies,Eagles,Bogeys,Doubles,Place Median
0,AlphaWired,364.8,4.0,114.0,232.5,16.0,-21.0,-3.0,21.0
1,New Team 4,354.0,3.5,102.0,223.5,20.0,-20.5,-3.5,19.0


In [42]:
# cuts made histogram

px.histogram(df,
            x='cuts_made',
            text_auto=True,
            title='Cuts Made Distribution',
            template='plotly_white')

In [43]:
melted_finish_medians = finish_medians.melt(id_vars='team',value_vars=['Top Finisher','2nd','3rd','4th','5th','Worst Finisher'])


fin_place_scatter = px.scatter(melted_finish_medians,
          x='variable',
          y='value',
          color='team',
          color_discrete_map=team_color,
          template='plotly_white',
          labels={'value':'Median Finish','variable':''},).update_traces(marker_size=12)

fin_place_scatter.update_layout(title_x=.5,
                               legend=dict(
                                   title=None,
                                   orientation='h',
                                   x=0,
                                   y=1.3
                               )
                              )

In [44]:
# table goes with scatter above

round(df[df.team=='unit_circle'][['fin_1','fin_2','fin_3','fin_4','fin_5','fin_6']].describe()[1:])

,fin_1,fin_2,fin_3,fin_4,fin_5,fin_6
mean,11.0,20.0,28.0,46.0,54.0,64.0
std,10.0,11.0,16.0,16.0,15.0,8.0
min,1.0,4.0,4.0,24.0,31.0,50.0
25%,2.0,11.0,21.0,36.0,42.0,57.0
50%,10.0,24.0,27.0,46.0,56.0,67.0
75%,21.0,27.0,39.0,56.0,66.0,70.0
max,24.0,33.0,52.0,68.0,69.0,71.0


In [45]:
# add 'wins' column
merge = df.copy()
merge['win_loss'] = merge['win_loss'].astype('int')

win_count = merge.groupby('team')['win_loss'].sum()

df = df.merge(win_count, how='left',on='team').rename(columns={'win_loss_y':'wins'})

In [49]:
# df['win_loss'] = df['win_loss'].astype('category')

fig = px.scatter(df,
              x='bird_num',
              y='total_pts',
              trendline='ols'#,trendline_scope='overall',
              )

results = px.get_trendline_results(fig).px_fit_results.iloc[0].rsquared
print(f"R-Squared Value: {results:.2f}")

fig.show()

R-Squared Value: 0.89


In [72]:
px.scatter(df[['team','bird_num','win_loss_x','wins']].groupby('team',as_index=False)['bird_num','win_loss_x'].sum(),
          x='bird_num',
          y='win_loss_x',
          trendline='ols',trendline_scope='overall',
          color='team',
          title=f"Expected Wins based on Number of Birdies")

C:\Users\mikej\AppData\Local\Temp\ipykernel_8328\919725591.py:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [162]:
cuts_per_week = pd.DataFrame(df.groupby('team')[['cuts_made','total_pts']].sum())
cuts_per_week['cuts_per_week'] = cuts_per_week['cuts_made'] / 8
px.bar(cuts_per_week[['cuts_per_week','total_pts']].sort_values(by='cuts_per_week',ascending=False).round(1),
#       log_y=True,
      text_auto=True,
      template='plotly_white',
      color='total_pts',
      height=350,
      color_continuous_scale=px.colors.sequential.Blues)

In [ ]:
### PLAYGROUND BELOW HERE ###

In [82]:
temp = df.groupby('team')['pars_pts','bird_pts','eag_pts','total_pts'].sum()
temp['bird_eagle_pts'] = temp.pars_pts + temp.bird_pts + temp.eag_pts
temp['portion'] = round(temp.bird_eagle_pts / temp.total_pts,2)
temp

C:\Users\mikej\AppData\Local\Temp\ipykernel_8328\3213610367.py:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,pars_pts,bird_pts,eag_pts,total_pts,bird_eagle_pts,portion
team,,,,,,
AlphaWired,909.0,1974,136,3005.0,3019.0,1.00
New Team 4,798.5,1854,184,2825.0,2836.5,1.00
Philly919,892.5,1890,144,2829.5,2926.5,1.03
Putt Pirates,940.5,2229,232,3442.5,3401.5,0.99
Snead's Foot,863.5,1788,184,2713.0,2835.5,1.05
Team Gamble,866.5,2130,232,3245.0,3228.5,0.99
txmoonshine,885.5,2106,160,3092.0,3151.5,1.02
unit_circle,972.5,2205,208,3438.5,3385.5,0.98


In [87]:
df.groupby('team').plc_pts.sum()

team
AlphaWired      179
New Team 4      182
Philly919       110
Putt Pirates    237
Snead's Foot     84
Team Gamble     234
txmoonshine     155
unit_circle     254
Name: plc_pts, dtype: int64

In [142]:
# percentage of time players make cut
df['cuts_per_start'] = df.cuts_made / df.players_started
df.groupby('team')['cuts_per_start'].mean().round(2)

team
AlphaWired      0.72
New Team 4      0.72
Philly919       0.70
Putt Pirates    0.79
Snead's Foot    0.63
Team Gamble     0.71
txmoonshine     0.71
unit_circle     0.83
Name: cuts_per_start, dtype: float64